In [ ]:
#| default_exp config

In [ ]:
%load_ext autoreload
%autoreload 2

# Configuration management

Functions for loading and managing experiment configurations.

In [ ]:
#| export
import yaml
from pathlib import Path
from typing import Dict, Any, Union
from dataclasses import dataclass

In [ ]:
#| export
@dataclass
class ExperimentConfig:
    """Configuration for a linguistic annotation experiment."""
    name: str
    date: str
    description: str
    input_dir: str
    pattern: str
    prompt_file: str
    output_dir: str
    model_provider: str
    model_name: str
    target_verb: str = None
    target_file: str = None
    batch_size: int = 10
    max_retries: int = 3
    metrics_config: Dict[str, Dict[str, str]] = None

In [ ]:
#| export
def load_config(config_path: Union[str, Path]) -> ExperimentConfig:
    """Load experiment configuration from a YAML file.
    
    Args:
        config_path: Path to the YAML configuration file
        
    Returns:
        ExperimentConfig object with loaded settings
    """
    with open(config_path, 'r') as f:
        config_data = yaml.safe_load(f)
    
    # Flatten nested structure
    experiment = config_data['experiment']
    data = config_data['data']
    output = config_data['output']
    model = config_data['model']
    processing = config_data.get('processing', {})
    metrics = config_data.get('metrics', {})
    
    return ExperimentConfig(
        name=experiment['name'],
        date=experiment['date'],
        description=experiment['description'],
        input_dir=data['input_dir'],
        pattern=data['pattern'],
        prompt_file=data['prompt_file'],
        output_dir=output['dir'],
        model_provider=model['provider'],
        model_name=model['name'],
        target_verb=processing.get('target_verb'),
        target_file=processing.get('target_file'),
        batch_size=processing.get('batch_size', 10),
        max_retries=processing.get('max_retries', 3),
        metrics_config=metrics.get('variables', {})
    )

In [ ]:
# Test configuration loading

try:
    config = load_config('../nbs/experiments/config/2025-06-23.yaml')
    print(f"✓ Loaded config: {config.name}")
    print(f"  Model: {config.model_name}")
    print(f"  Target: {config.target_verb}")
    
    # Test basic validation
    assert config.name is not None, "Config name should not be None"
    assert config.model_name is not None, "Model name should not be None"
    print("✓ Configuration validation passed")
    
except FileNotFoundError:
    print("⚠ Config file not found - this is expected in test environment")
except Exception as e:
    print(f"✗ Config test failed: {e}")
    raise
